In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import numpy as np

# Step 2: Load data
file_1_path = "/content/Dataset_CO.xlsx"
file_2_path = "/content/Blank_CIs.xlsx"

# Load the files
df_train = pd.read_excel(file_1_path)
df_test = pd.read_excel(file_2_path)

# Standardize column names for both datasets
df_train.columns = df_train.columns.str.strip()
df_test.columns = df_test.columns.str.strip()

# Compare column names explicitly
print("Columns in Dataset_CO after stripping:", df_train.columns.tolist())
print("Columns in Dataset_Blank after stripping:", df_test.columns.tolist())

# Check for mismatched columns
common_columns = set(df_train.columns).intersection(df_test.columns)
print("Common columns:", common_columns)

# Step 3: Analyze dataset
print("Dataset_CO (First few rows):")
print(df_train.head())

print("\nDataset_Blank (First few rows):")
print(df_test.head())

# Step 4: Filter CIs with >50 tickets
ci_counts = df_train['Cis'].value_counts()
valid_cis = ci_counts[ci_counts > 50].index
df_train['Cis'] = df_train['Cis'].apply(lambda x: x if x in valid_cis else "Unknown")

# Step 5: Prepare training data
# Combine text fields for model input and handle missing values
df_train['text'] = (
    df_train['Sd'].fillna('') + " " +
    df_train['Resolution_note'].fillna('') + " " +
    df_train['Resolution_code'].fillna('')
)
X = df_train['text']
y = df_train['Cis']

# Step 6: Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Define and train model pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000, stop_words='english')),
    ('clf', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)

# Evaluate model
y_pred = pipeline.predict(X_val)
print("\nModel Performance on Validation Set:")
print(classification_report(y_val, y_pred))

# Step 8: Predict on Dataset_Blank
# Combine text fields for prediction and handle missing values
df_test['text'] = (
    df_test['Sd'].fillna('') + " " +
    df_test['Resolution_note'].fillna('') + " " +
    df_test['Resolution_code'].fillna('')
)
df_test['Cis'] = pipeline.predict(df_test['text'])

# Replace infrequent CIs with "Unknown"
df_test['Cis'] = df_test['Cis'].apply(lambda x: x if x in valid_cis else "Unknown")

# Step 9: Save the updated Dataset_Blank
output_path = "/content/Dataset_Blank_Filled.xlsx"
df_test.to_excel(output_path, index=False)
print(f"Updated Dataset saved to {output_path}")


Columns in Dataset_CO after stripping: ['Number', 'Created', 'Resolved', 'State', 'Assignment group', 'Sd', 'Cis', 'Resolution_note', 'Priority', 'Category', 'Subcategory', 'Channel', 'Major incident state', 'Resolution Category', 'Resolution Sub Category', 'Resolution_code', 'Actions taken', 'Work notes', 'Location', 'Incident state', 'Reported for', 'OLD_Reported for', 'Caller', 'Description', 'Company', 'Product Vendor', 'Product Vendor Ticket', 'Vendor Comments', 'Resolve time', 'Resolved by', 'Change Request', 'Caused by Change', 'Closed', 'Closed by', 'Comments and Work notes', 'Created by', 'Parent Incident', 'Problem', 'Severity', 'Ticket Status Summary']
Columns in Dataset_Blank after stripping: ['Number', 'Created', 'Resolved', 'State', 'Assignment group', 'Sd', 'Cis', 'Resolution_note', 'Priority', 'Category', 'Subcategory', 'Channel', 'Major incident state', 'Resolution Category', 'Resolution Sub Category', 'Resolution_code', 'Actions taken', 'Work notes', 'Location', 'Inci

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Updated Dataset saved to /content/Dataset_Blank_Filled.xlsx
